In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
			)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__

live = client.is_live()
connected = client.is_connected()

print(f"Weaviate client is ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"live: {live}")
print(f"Connected: {connected}")

In [ ]:
# How to create a collection with two named vectors
from weaviate.classes.config import Configure, Property, DataType

client.collections.create(
    "twonamedvectors",
    vectorizer_config=[
        # Set a named vector
        Configure.NamedVectors.text2vec_openai(
            name="NamedVectorOne",
            source_properties=["one"]       
        ),
        # Set another named vector
        Configure.NamedVectors.text2vec_openai(
            name="NamedVectorTwo",
            source_properties=["two"]       
        )
    ],
    properties=[  # Define properties
        Property(name="one", data_type=DataType.TEXT),
        Property(name="two", data_type=DataType.TEXT)
    ]
)


In [ ]:
# How to insert data into the collection with named vectors
article = client.collections.get("twonamedvectors")
uuid = article.data.insert(
    {
        "one": "this is for one",
        "two": "this is for two",
    }
)

In [ ]:
# Fetch the object with the named vectors
collection = client.collections.get("<COLLECTION_NAME>")
response = collection.query.fetch_objects()
vector_names = ["<NAMED_VECTOR_PROP>", "NAMED_VECTOR_PROP"]

data_object = collection.query.fetch_object_by_id(
    uuid="<UUID>", 
    include_vector=vector_names
)

print(data_object)


In [ ]:
# Querting with named vectors
from weaviate.classes.query import MetadataQuery

col = client.collections.get("<COLLECTION_NAME>")
response = col.query.near_text(
    query="<QUERY>",
    target_vector="NAMED_VECTOR_PROP",
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)